In [2]:
import findspark
findspark.init()
findspark.find()

import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import pyspark.sql.functions as func
from pyspark.sql import Window
from pyspark.sql.types import *
import re
import time

In [3]:
# Criando sessao no spark

spark = SparkSession.builder \
    .master("local[1]") \
    .appName("Debate Tweets") \
    .config("spark.sql.debug.maxToStringFields", 2000) \
    .getOrCreate()

# Carregando dataframe
df = spark.read.\
    format("com.databricks.spark.csv") \
    .option("delimiter", "\t") \
    .load("./debate-tweets-001.tsv")

22/12/11 17:44:05 WARN Utils: Your hostname, ip-172-31-77-113 resolves to a loopback address: 127.0.1.1; using 172.31.77.113 instead (on interface eth0)
22/12/11 17:44:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/11 17:44:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
columns =  ("id"
            ,"content"
            ,"coordinate_0"
            ,"coordiante_1"
            ,"retweeted"
            ,"tweet_id"
            ,"tweet_id_str"
            ,"created_at"
            ,"created_at_str"
            ,"in_reply_to_user_id"
            ,"lang"
            ,"place_name"
            ,"place_id"
            ,"place_bb_00"
            ,"place_bb_01"
            ,"place_bb_10"
            ,"place_bb_11"
            ,"place_bb_20"
            ,"place_bb_21"
            ,"place_bb_30"
            ,"place_bb_31"
            ,"place_type"
            ,"place_country_code"
            ,"place_country"
            ,"place_full_name"
            ,"user__screen_name"
            ,"user_id"
            ,"user_id_str"
            ,"user_location"
            ,"user_friend_count"
            ,"user_created_at"
            ,"user_name")

df = df.toDF(*columns)

### 1. Quais foram as hashtags mais usadas pela manhã, tarde e noite? 

In [44]:
# Foi necesssária a criação de uma UDF function visto que um tweet pode conter mais de uma '#' e o método regex_extract seleciona apenas a PRIMEIRA ocorrência do regex.

def extract(s):
    all_matches = re.findall(r'#\w+', s)
    return all_matches

extract_hashtags = func.udf(extract, ArrayType(StringType()))

# .filter: fará com que apenas sentenças com # sejam selecionadas
# .withcolumn: cria a coluna 'extracted', que é uma lista com as hashtags encontradas em cada tweet usando a funcao extract_hashtags
# .select: seleciona
#       1. data de criação
#       2. .explode_outer: para transformar cada item da coluna 'extracted' em uma nova linha na coluna hashtag

df_hashtags = df \
    .filter(func.col('content').rlike('#\w+')) \
    .withColumn('extracted', extract_hashtags('content')) \
    .select('created_at', func.explode_outer(func.col('extracted')).alias('hashtag') )


#### Período da Manhã

In [45]:
# .filter: filtra linhas que contenham substring no campo 'created_at' com horarios entre 6 e 12 horas
# .groupBy: agrupa hashatags repetidas
# .count: cria coluna 'count' com contador da agrupação
# .sort: ordena dataframe usando a coluna count
# .show: mostra o dataframe

df_hashtags \
    .filter(func.col('created_at').substr(12,2).between(6,11)) \
    .groupBy('hashtag') \
    .count() \
    .sort('count', ascending=0) \
    .show(n=20)


+--------------------+-----+
|             hashtag|count|
+--------------------+-----+
|   #EMABiggestFans1D|14818|
|#EMABiggestFansJu...|14367|
|             #trndnl|  604|
|#VoteVampsTeenAwards|  365|
|             #bomdia|  317|
|         #QueroNoTVZ|  315|
|             #BomDia|  306|
|              #TwOff|  282|
|#EMABiggestFansJu...|  231|
|      #QuandoEuCasar|  146|
+--------------------+-----+
only showing top 10 rows



#### Período da Tarde

In [14]:
# .filter: filtra linhas que contenham substring no campo 'created_at' com horarios entre 6 e 12 horas
# .groupby: agrupa hashatags repetidas
# .count: cria coluna 'count' com contador da agrupação
# .sort: ordena dataframe usando a coluna count
# .show: mostra o dataframe

df_hashtags \
    .filter(func.col('created_at').substr(12,2).between(12,18)) \
    .groupBy('hashtag') \
    .count() \
    .sort('count', ascending=0) \
    .show(n=20)

+--------------------+-----+
|             hashtag|count|
+--------------------+-----+
|   #EMABiggestFans1D|70859|
|#EMABiggestFansJu...|60128|
|        #StealMyGirl| 5704|
|         #QueroNoTVZ| 5026|
|   #bigpaynodanceoff| 1514|
|  #AustinMahoneChile| 1230|
|       #AustinMahone| 1111|
|#demiyourstorydoe...| 1090|
|             #trndnl| 1049|
|    #HottieOfTheWeek|  925|
+--------------------+-----+
only showing top 10 rows



#### Período da Noite

In [ ]:
# .filter: filtra linhas que contenham substring no campo 'created_at' com horarios entre 19 e 23 horas ou 0 e 5 horas
# .groupby: agrupa hashatags repetidas
# .count: cria coluna 'count' com contador da agrupação
# .sort: ordena dataframe usando a coluna count
# .show: mostra o dataframe

df_hashtags \
    .filter(func.col('created_at').substr(12,2).between(19,23) | func.col('created_at').substr(12,2).between(0,5)) \
    .groupBy('hashtag') \
    .count() \
    .sort('count', ascending=0) \
    .dropna() \
    .show(n=20)

+--------------------+------+
|             hashtag| count|
+--------------------+------+
|#EMABiggestFansJu...|133599|
|   #EMABiggestFans1D|129768|
|       #camilasayshi| 10507|
|        #DebateNoSBT|  3416|
|    #CartersNewVideo|  3207|
|   #bigpaynodanceoff|  2878|
|     #TheVoiceBrasil|  2819|
|          #Vote5HEMA|  2666|
|        #AssistamODR|  2545|
|     #DebateNaRecord|  2494|
+--------------------+------+
only showing top 10 rows



### 2. Quais as hashtags mais usadas em cada dia?

In [18]:
# Habilita formato de data com EEE
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY") 

# Window criada para usar filtro de máximo entre os grupos
w = Window.partitionBy('timestamp') 


# .dropna: exclui linhas com valores nulos
# .select: seleciona as colunas:
#    1. hashtag
#    2. cria coluna timestampo com base na coluna created_at
# .groupBy: agrupa por timestamp de criação e hashtag
# .count: cria coluna 'count' com contador da agrupação
# .sort: ordena dataframe usando a coluna count
# .withColumn: cria coluna max, que terá a quantidade maxima de cada grupo
# .where: filtra apenas os maximos
# .drop: exclui coluna com os maximos
# .show: mostra o datafram

df_hashtags \
    .dropna() \
    .select('hashtag', func.to_timestamp(func.col('created_at'), 'EEE MMM dd HH:mm:ss ZZZZ yyyy').substr(0,10).alias('timestamp')) \
    .groupBy( 'timestamp','hashtag') \
    .count() \
    .sort('count', ascending=0) \
    .withColumn('max', func.max('count').over(w)) \
    .where(func.col('count') == func.col('max')) \
    .drop('max') \
    .show(truncate=0)


+----------+---------------------------+-----+
|timestamp |hashtag                    |count|
+----------+---------------------------+-----+
|null      |#EMABiggestFansJustinBieber|7    |
|2014-10-15|#EMABiggestFans1D          |34547|
|2014-10-16|#EMABiggestFans1D          |68633|
|2014-10-17|#EMABiggestFansJustinBieber|49488|
|2014-10-18|#EMABiggestFansJustinBieber|27454|
|2014-10-19|#EMABiggestFansJustinBieber|33582|
|2014-10-20|#EMABiggestFansJustinBieber|10452|
+----------+---------------------------+-----+



#### 3. Qual o número de tweets por hora a cada dia?

In [42]:
# contar quantidade de tweets por dia
# agrupar por dia
# dividir quantidade por 24

# Habilita formato de data com EEE
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY") 

# .select: seleciona coluna com timestamp criado a partir da coluna created_at
# .groupBy: agrupa pela data
# .agg: cria coluna 'count' que é um contador de tweets por dia
# .withColumn: cria coluna 'number of tweets' com quantidade de tweets por hora a cada dia (count / 24)
# .drop: exclui coluna de contador geral
# .show: mostra o datafram

df_hashtags \
    .dropna() \
    .select(func.to_timestamp(func.col('created_at'), 'EEE MMM dd HH:mm:ss ZZZZ yyyy').substr(0,10).alias('timestamp')) \
    .groupBy( 'timestamp') \
    .count() \
    .withColumn('number of tweets', func.col('count') / 24) \
    .drop('count') \
    .show(truncate=0)


+----------+------------------+
|timestamp |number of tweets  |
+----------+------------------+
|2014-10-15|6041.916666666667 |
|2014-10-16|12244.208333333334|
|2014-10-17|10139.125         |
|2014-10-18|7554.708333333333 |
|2014-10-19|9477.791666666666 |
|2014-10-20|3134.625          |
+----------+------------------+



#### 4. Quais as principais sentenças relacionadas à palavra “Dilma”?

In [49]:
# .select: seleciona apenas a coluna content
# .fiter: filtra a coluna content com expressões que deem match com a palabra Dilma
# .groupBy: agrupa dados pela coluna content
# .count: cria coluna 'count' com contage baseado no goupby
# .sort: ordena de forma decrescente
# .show: mostra o dataframe
df \
    .select('content') \
    .filter(func.col('content').rlike('Dilma')) \
    .groupBy('content') \
    .count() \
    .sort('count', ascending=0) \
    .show(truncate=0, n = 20)

#### 5. Quais as princiapais sentenças relacionadas à palavra "Aécio"?

In [ ]:
# .select: seleciona apenas a coluna content
# .fiter: filtra a coluna content com expressões que deem match com a palabra Dilma
# .groupBy: agrupa dados pela coluna content
# .count: cria coluna 'count' com contage baseado no goupby
# .sort: ordena de forma decrescente
# .show: mostra o dataframe
df \
    .select('content') \
    .filter(func.col('content').rlike('Aécio')) \
    .groupBy('content') \
    .count() \
    .sort('count', ascending=0) \
    .show(truncate=0, n = 20)